In [137]:
import numpy as np
import pandas as pd

In [138]:
import json
!conda install -c conda-forge geopy --yes
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: ...working... done

# All requested packages already installed.



In [41]:
!conda install -c conda-forge folium --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


Loading the dataframe df_toronto from the previous part

In [195]:
df_toronto = pd.read_csv('Week3-df_toronto.csv')
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Explore and cluster the neighborhoods in Toronto. For simplicity, let's simplify the above map and segment and cluster only the neighborhoods in Downtown Toronto. So let's slice the original dataframe and create a new dataframe of the Downtown data.

In [180]:
downtown_df = df_toronto.loc[df_toronto['Borough']=="Downtown Toronto"].reset_index(drop=True)
downtown_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


Getting geographical coordinates of Downtown Toronto

In [181]:
address = "Downtown Toronto, Toronto, Ontario, Canada"
geolocator = Nominatim(user_agent = "toronto_explorer", timeout = 5)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


Visualizing neighbourhoods in downtown Toronto

In [182]:
# create map of downtown Toronto using latitude and longitude values
map_downtown = folium.Map(location = (latitude, longitude), zoom_start = 10)

# add markers to map
for lat, lng, neigh in zip(downtown_df['Latitude'], downtown_df['Longitude'], downtown_df['Neighbourhood']):
    label = folium.Popup(neigh, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Define Foursquare Credentials and Versions 

In [183]:
CLIENT_ID = 'Y1HPCAKL0ATYGLZUT3XISKQNEKPR0ILGDJBFWDAKTXX2TDAY' # your Foursquare ID
CLIENT_SECRET = '4NBMSQFDJL0GF5JQZ5WLM2P2ZE40OL2N4NTUEFVFP2P2E41S' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y1HPCAKL0ATYGLZUT3XISKQNEKPR0ILGDJBFWDAKTXX2TDAY
CLIENT_SECRET:4NBMSQFDJL0GF5JQZ5WLM2P2ZE40OL2N4NTUEFVFP2P2E41S


Exploring neighbourhoods in downtown Toronto. We will get the top 100 venues that are in each neighbourhood within a radius of 600 meters.

In [248]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [249]:
downtown_venues = getNearbyVenues(names = downtown_df['Neighbourhood'], latitudes = downtown_df['Latitude'], 
                                  longitudes = downtown_df['Longitude'], LIMIT = 100)
downtown_venues.head()

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
1,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


Couting how many nearby venues in each neighbourhood

In [250]:
downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,92,92,92,92,92,92
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",19,19,19,19,19,19
"Cabbagetown, St. James Town",59,59,59,59,59,59
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,18,18,18,18,18,18
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [251]:
print("There are {} unique categories for nearby venues".format(len(downtown_venues["Venue Category"].unique())))

There are 210 unique categories for nearby venues


Analyzing each neighbourhood. We apply One Hot Encoding.

In [252]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
downtown_onehot = pd.concat([downtown_onehot.iloc[:,-1], downtown_onehot.iloc[:,:-1]], axis = 1)

downtown_onehot.shape

(1435, 211)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [253]:
downtown_frequency = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_frequency

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02000,0.000000,...,0.00,0.000000,0.00,0.020000,0.000000,0.00,0.00,0.01,0.00,0.000000
1,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,0.000000,...,0.00,0.000000,0.00,0.021739,0.000000,0.00,0.00,0.00,0.00,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.052632,0.052632,0.052632,0.105263,0.157895,0.052632,0.00000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
3,"Cabbagetown, St. James Town",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.016949
4,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01000,0.000000,...,0.00,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.01,0.00,0.010000
5,"Chinatown, Grange Park, Kensington Market",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00,0.000000,0.01,0.030000,0.000000,0.00,0.03,0.01,0.00,0.010000
6,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000
7,Church and Wellesley,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01000,0.000000,...,0.00,0.000000,0.00,0.000000,0.000000,0.01,0.01,0.00,0.01,0.020000
8,"Commerce Court, Victoria Hotel",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04000,0.000000,...,0.01,0.000000,0.00,0.020000,0.000000,0.00,0.00,0.01,0.00,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04000,0.000000,...,0.01,0.000000,0.00,0.010000,0.000000,0.00,0.00,0.01,0.00,0.000000


Top 5 venues in each neighbourhood

In [254]:
num_top_venues = 5

for neigh in downtown_frequency['Neighbourhood']:
    print("----"+neigh+"----")
    temp = downtown_frequency[downtown_frequency['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:] # To ignore the first row since it contains no frequency
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Steakhouse  0.04
3   Restaurant  0.03
4  Salad Place  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1           Restaurant  0.05
2   Seafood Restaurant  0.03
3                 Café  0.03
4  Japanese Restaurant  0.03


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Airport Service  0.16
1          Coffee Shop  0.11
2       Airport Lounge  0.11
3        Boat or Ferry  0.11
4  Rental Car Location  0.05


----Cabbagetown, St. James Town----
         venue  freq
0   Restaurant  0.08
1  Coffee Shop  0.08
2  Pizza Place  0.05
3         Park  0.05
4    Gastropub  0.03


----Central Bay Street----
              venue  freq
0       Coffee Shop  0.13
1               Bar  0.03
2      Burger Joint  0.03
3    Ice Cream Shop  0.03
4  Sushi Restaurant  0.03


---

In [289]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_frequency['Neighbourhood']

for ind in np.arange(downtown_frequency.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_frequency.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Salad Place,Theater
1,Berczy Park,Coffee Shop,Restaurant,Hotel,Japanese Restaurant,Café
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Boat or Ferry,Coffee Shop,Airport Lounge,Boutique
3,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Park,Pizza Place,Bakery
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Sandwich Place
5,"Chinatown, Grange Park, Kensington Market",Bar,Café,Chinese Restaurant,Dumpling Restaurant,Dessert Shop
6,Christie,Grocery Store,Café,Park,Playground,Nightclub
7,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Restaurant,Hotel,American Restaurant


Run K-Means to cluster the neighbourhoods into 5 clusters

In [290]:
# set number of clusters
kclusters = 4

downtown_clustering = downtown_frequency.drop('Neighbourhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:num_top_venues] 

array([1, 1, 3, 1, 1])

In [291]:
downtown_clustering.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02000,0.0,0.00000,...,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.01,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,0.0,0.01087,...,0.0,0.0,0.0,0.021739,0.0,0.0,0.0,0.00,0.0,0.000000
2,0.0,0.052632,0.052632,0.052632,0.105263,0.157895,0.052632,0.00000,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.016949
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01000,0.0,0.00000,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.010000


In [292]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge downtown_df and dowtown_clustering add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Dim Sum Restaurant,Falafel Restaurant
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Restaurant,Coffee Shop,Park,Pizza Place,Bakery
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Theater
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Tea Room
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Hotel,Restaurant,Bakery
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Restaurant,Hotel,Japanese Restaurant,Café
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Sandwich Place
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1,Coffee Shop,Café,Steakhouse,Salad Place,Theater
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,1,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant


Visualize

In [321]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

In [294]:
downtown_cluster_1 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

downtown_cluster_1




,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Rosedale,0,Park,Playground,Trail,Dim Sum Restaurant,Falafel Restaurant


In [295]:
downtown_cluster_2 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

downtown_cluster_2

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"Cabbagetown, St. James Town",1,Restaurant,Coffee Shop,Park,Pizza Place,Bakery
2,Church and Wellesley,1,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café
3,Harbourfront,1,Coffee Shop,Pub,Bakery,Park,Theater
4,"Ryerson, Garden District",1,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Tea Room
5,St. James Town,1,Coffee Shop,Café,Hotel,Restaurant,Bakery
6,Berczy Park,1,Coffee Shop,Restaurant,Hotel,Japanese Restaurant,Café
7,Central Bay Street,1,Coffee Shop,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Sandwich Place
8,"Adelaide, King, Richmond",1,Coffee Shop,Café,Steakhouse,Salad Place,Theater
9,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant
10,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant


In [296]:
downtown_cluster_3 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

downtown_cluster_3

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
18,Queen's Park,2,Pharmacy,Grocery Store,Shopping Mall,Bank,Café


In [297]:
downtown_cluster_4 = downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

downtown_cluster_4

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",3,Airport Service,Boat or Ferry,Coffee Shop,Airport Lounge,Boutique


Based on this clustering, we suggest that
- Cluster 1: Rosedale neighbourhood with many playground/outdoor areas (perhaps it is ideal and friendly for families)
- Cluster 2: Center Areas (i.e., Central Business Districts)
- Cluster 3: Queen's Park (a resendential area)
- Cluster 4: Areas around the Billy Bishop Toronto Airport